In [1]:
# pip install pymorphy2 >> None

In [2]:
import pandas as pd
import numpy as np
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# задаем директорию
file_path = 'C:/Users/user/Desktop/Hackathons/LinkedIn_menthoring/'

In [4]:
# открываем файлы с данными
posts = pd.read_csv(os.path.join(file_path, 'posts.csv'), index_col=0)
profiles = pd.read_csv(os.path.join(file_path, 'profiles.csv'), index_col=0)

In [5]:
# открываем датафрейм с постами
posts.head(2)

,user_id,text,likes,comments,reposts
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [6]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [7]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [8]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [9]:
# удалим посты на украинском языке

# определяем шаблон для украинских символов (по специфичным для данного языка символам)
ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# создаем маску, указывающую строки, в которых столбец "post" содержит текст на украинском языке
mask = posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)

# сохраняем в датафрейме только строки, в которых маска имеет значение False
posts = posts[~mask]

In [10]:
# сохраняем хэштэги в отдельный столбец перед их удалением из постов
posts['hashtags'] = posts['post'].str.findall(r'#([^\s]+)').apply(lambda x: ', '.join(x))

In [11]:
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

In [12]:
# удаляем слова, которые идут после хэш-тэга
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'#[^\s]+', ' ', x))

In [13]:
# производим замену дефиса на пробел
posts["post_lemmatized"] = posts["post_lemmatized"].str.replace("-", " ")

In [14]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)
# только русские буквы и пробелы
posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', ' ', regex=True) 

In [15]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
file_path_words = os.path.join(file_path, 'stop_words.txt')
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [17]:
'''если НЕ оставляем в постах английские слова'''
# определяем шаблон регулярного выражения для русских букв
pattern = '[^а-яА-ЯёЁ]'
# создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
# фильтруем датафрейм, используя маску
posts = posts[mask]

In [18]:
posts.head()

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0,"podcast, миражи",говоря подкаст мираж доступный соцсеть вконтакте разобраться прикрутить
2,ali-wodan,"Хэй честной народ! Ищу векторного иллюстратора на проект, с возможным длительным сотрудничеством по итогу. Можно удаленно. Уровень иллюстраций не хуже чем в примере https://lnkd.in/gkrvhxYРепост, пожалуйста)Если ты умеешь так рисовать, напиши мне в личку :-)Hey! I am looking for an Illustrator who can help us with a project! You need to have skills like in the example below or upper. Text me to discuss prices and due :-)#вакансия #vacancy #illustration #иллюстрация #project #проект",6,0,0,"вакансия, vacancy, illustration, иллюстрация, project, проект",честной народ искать векторный иллюстратор возможный длительный сотрудничество итогу удаленно уровень иллюстрация уметь рисовать
3,ali-wodan,"Новый пост подкаста ""Миражи"" на всех платформах:В аудио: https://lnkd.in/giWuSVNВ инстаграме: https://lnkd.in/gbV6yxKН #automotive # # #cars #а патреоне: https://lnkd.in/gfShqTU#саморазвитие #психология #психологиячеловека #психологияжизни #психологияличности #личностныйрост #подкаст #подкастмиражи #маркетинг #осознанность #мысли #мысливслух #установки #влияние #подсознаниеИспользованные звуки, музыка и картинки:Аудиоцитата из фильма ""Формула любви"" Марка ЗахароваJardins du Luxembourg by Jahzzar is licensed under a Attribution-ShareAlike 3.0 International License.prisoner by Luis Prado from the Noun Project",1,0,0,"automotive, cars, а, саморазвитие, психология, психологиячеловека, психологияжизни, психологияличности, личностныйрост, подкаст, подкастмиражи, маркетинг, осознанность, мысли, мысливслух, установки, влияние, подсознаниеИспользованные",подкаст миражи платформах аудио инстаграме патреоне звуки музыка картинки аудиоцитат фильм формула любви марк захарова
4,ali-wodan,"Подкаст Миражи, Эпизод 13Ошибка невозвратных затратpodcast.ru/1539345144#подкаст #podcast #it #podcasts #psychology #психология #miragespodcast",2,0,0,"подкаст, podcast, it, podcasts, psychology, психология, miragespodcast",подкаст миражи эпизод ошибка невозвратный затрат
5,ali-wodan,Новый эпизод об эффекте ИКЕА на всех платформах podcast.ru/1539345144,1,0,0,,эпизод эффект икеа платформа


In [19]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2057 entries, 0 to 8597
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          2057 non-null   object
 1   post             2057 non-null   object
 2   likes            2057 non-null   object
 3   comments         2057 non-null   int64 
 4   reposts          2057 non-null   int64 
 5   hashtags         2057 non-null   object
 6   post_lemmatized  2057 non-null   object
dtypes: int64(2), object(5)
memory usage: 128.6+ KB


Объединяем датафреймы

In [20]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [21]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [22]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [23]:
# удаляем из столбца likes точки, запятые и пробелы
df["likes"] = df["likes"].replace(r'\.|\,|\s', '', regex=True)

# меняем тип данных сотлбца likes на integer
df["likes"] = df["likes"].astype("int64")

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2048 entries, 0 to 2056
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           2048 non-null   object
 1   post              2048 non-null   object
 2   likes             2048 non-null   int64 
 3   comments          2048 non-null   int64 
 4   reposts           2048 non-null   int64 
 5   hashtags          2048 non-null   object
 6   post_lemmatized   2048 non-null   object
 7   user_name         2048 non-null   object
 8   user_head         2048 non-null   object
 9   user_work         1960 non-null   object
 10  user_position     2048 non-null   object
 11  user_tags         392 non-null    object
 12  user_location     2041 non-null   object
 13  user_viewers      2048 non-null   object
 14  user_contacts     2048 non-null   object
 15  user_common_info  1861 non-null   object
dtypes: int64(3), object(13)
memory usage: 272.0+ KB


In [25]:
# Сохраняем датафрейм
df.to_csv(os.path.join(file_path, 'linkedin.csv'))

# Сохраняем датафрейм лемматизации
df[[
    'user_id', 'post_lemmatized', 'likes', 'comments', 'reposts'
]].to_csv(os.path.join(file_path, 'post_lemmatized.csv'))

In [26]:
df.sample(5)

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
1136,ferrethikky,"Глубинное интервью.Изучаем прошлый опыт и мотивацию пользователей, чтобы выявить проблемы и узнать о возможностях для нового продукта и сервиса.#design #designinspiration",2,0,0,"design, designinspiration",глубинный интервью изучать опыт мотивация пользователей выявить проблема возможность продукт сервиса,Andrey Rozhnov,Product Designer | User Interface Designer with 2+ years | Banking | IT software development | Cryptocurrency | Mobile and Web Apps,Stoloto,Product Designer,NaN,"Tbilisi, Georgia",802,500+,"My work experience started with graphic design. At first I created logos and presentations for the companies. Soon I decided to learn a web design, rendering promo pages and landings.My great growth was obtained in the IT-startup, where I made sure I want to work as a User Interface Designer. Starting soon, I honed my abilities and skills in creating UI-kit and components for the big products.Then I had another project in the banking sector. My level of responsibilities and tasks has significantly increased. I learned metrics to improve user interaction with the product and also provided support and improvement of the current UI-kit.Overall, the main development of my career was a transition to a Product Designer. This work includes identification of problems in the product and offering solutions to improve it, based on the needs of the business strategy. Moreover, I use feedback from users by research.The software I use:1. Products from Adobe2. Figma3. Miro4. Google Products"
1975,ulugbekalimov,UI/UX исследованиеКакой лучше?Обратная связь ценится.#ux #ui #research #figma,983,35,0,"ux, ui, research, figma",исследованиекака связь ценится,Улугбек Алимов,CEO Founder - Motion Box,Davr Bank,Design Lead,NaN,"Tashkent, Tashkent, Uzbekistan","1,981",500+,👋 Welcome!Всем привет! Я UI/UX designer и motion designer. В активном поиске интересных проектов. Слежу за всеми трендами дизайна. Работаю ответственно и в срок. Готов предложить свое видение.Telegram: https://t.me/byulugbekFacebook: https://www.facebook.com/byulugbek/ Dribbble: https://dribbble.com/byulugbek
369,vladimir-yelizaroff-575b6884,"Если вы за все года которые прошли с нашей молодости, не добились большего нежели жалкое существование ВЫ либо не узнали законов по которым строится благосостояние либо просто не следовали им. https://lnkd.in/ejme4GZ",0,0,0,,молодости добиться нежели жалкий существование либо строиться благосостояние либо следовать,Vladimir Yelizaroff,Software Development,StecPoint,Director Of Customer Service,NaN,"Moscow, Moscow, Russia","1,378",500+,"We create highly loaded platforms, design web and mobile applications, online services and marketplaces on open-source software. We rewrite old software for microservices and OpenSource. We provide ready-made teams and help to avoid dependence on foreign IT-products.Создаем высоконагруженные платформы, проектируем web и мобильные приложения, интернет-сервисы и маркетплейсы на открытом ПО. Переписываем старый софт на микросервисы и OpenSource. Предоставляем готовые команды и помогаем избежать зависимости от зарубежных IT-продуктов."
433,melekes,Находки в сети за октябрь,0,0,0,,находка сеть,Anton Kalyaev,"Software Engineer, Traveler, Glorified plumber #distributedsystems #consensus",Parity Technologies,Core Rust Developer,NaN,Russia,"1,183",500+,"Hi! My name is Anton. I am an experienced (10+ years) backend developer interested in distributed systems and hard problems. Ideally looking for a remote position, although relocation is possible.- “not a great programmer; just a good programmer with great habits” (Kent Beck)- have a blog http://homeonrails.com/. Popular articles include How to be productive withalmost any text editor https://www.homeonrails.com/2016/05/how-to-be-productive-with-almost-any-editor/ and Debugging the Me

In [27]:
filtered_df = df[df['post_lemmatized'].str.contains('обучение|ментор|менторство|менторинг|наставник|наставничество|подопечный', case=False)]

In [28]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131 entries, 77 to 2018
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           131 non-null    object
 1   post              131 non-null    object
 2   likes             131 non-null    int64 
 3   comments          131 non-null    int64 
 4   reposts           131 non-null    int64 
 5   hashtags          131 non-null    object
 6   post_lemmatized   131 non-null    object
 7   user_name         131 non-null    object
 8   user_head         131 non-null    object
 9   user_work         121 non-null    object
 10  user_position     131 non-null    object
 11  user_tags         26 non-null     object
 12  user_location     130 non-null    object
 13  user_viewers      131 non-null    object
 14  user_contacts     131 non-null    object
 15  user_common_info  113 non-null    object
dtypes: int64(3), object(13)
memory usage: 17.4+ KB


In [29]:
filtered_df.head()

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
77,agratoth,"Что едят единороги?Уже не первый год я по мере сил своих физических и ментальных оказываю менторскую помощь самым разным ребятам, от начинающих разработчиков (выбрать путь для профессионального развития, собрать роадмап для штудирования книг и т.д.) до HR (в этих наших стеках и фреймворках, как показывает практика, черт ногу сломит, если не вариться в этом постоянно). И неоднократно ребята спрашивали меня - почему я не делаю вебинаров по разным темамИ вот прошли годы, слово “вебинар” уже вышло из моды, заменившись на “воркшоп”, и я, наконец, созрел. А, с учетом на одном дыхании прочитанной нетленки Джеффа Уокера “Запуск!”, созрел даже сразу за деньги. Ну а почему бы и да?Итак, где-нибудь ориентировочно 17 или 18 сентября приглашаю всех заинтересованных на воркшоп. В течение четырех-пяти часов в прямом эфире я буду писать (и объяснять!) целый небольшой агротех-сервис. Да какой сервис! Это будет симулятор большой и взрослой ERP-системы, автоматизирующей деятельность фермы по выращиванию… единорогов. Ну, а почему бы и нет? Для единорогов нужно будет строить единорожни (ну, не конюшни же?), каждый из них будет обладать определенными статами, которые можно будет развивать скрещиванием. Для них надо будет закупать цветы и нектар (все ведь знают, чем питаются единороги?). Единорожий приплод можно будет продавать на внутреннем виртуальном рынке. Этакий небольшой бизнес-симулятор.В целом, за исключением контекста, система будет вполне серьезной, с бизнес-логикой, отложенными задачами и триггерами.Стек проекта:- на бэкенде - FastAPI, за ORM - ormar+alembic, за БД - PostgreSQL, за брокера - NATS, хранилище кэша - Redis, куда ж без него. - на фронтэнде - React 18 без CRA, Typescript, Redux-Toolkit. За UI будет отвечать Tailwind с плагинами. За сборку - Webpack с плагинами. Также есть Babel с плагинами, и PostCSS (угадайте с чем… правильно!), тоже с плагинами. - в инфраструктуре - Docker и nginx. Ansible и Gitlab CI/CD - если успеем.В целом, хочется сделать законченный полностековый проект, от верстки фронта до деплоя и девопсовых свистелок и … плейбуков. Возможно, в рамки одного воркшопа это все и не влезет, но мы будем очень стараться Во всяком случае, я почти обещаю сильно не тупить - проект то уже реализован, буду в него подглядывать, пока никто не видитТеперь про проведение. Проводить буду в формате стрима, в приватном канале youtube. Более вменяемых вариантов я не знаю, не на твич же с таким идти. Ну, может еще в ВК будет дубль трансляции, но это не точноНу и участие. Оно платное. Целых 150-200 рублей с человека! Ибо, как говорил Джокер - “Если ты что-то можешь сделать, не делай это бесплатно”. А годноты будет наваломВ целом, если идея вам зашла - то приветствуются лайки, комментарии и, конечно, репосты. Ну а если вы дочитали до этого места, и даже заинтересованы поприсутствовать на трансляции - смело пишите мне в телегу @agratoth, сделаем чатик и попытаемся все это организовать#react #fastapi #workshop #воркшоп_единороги",21,3,3,"react, fastapi, workshop, воркшоп_единороги",единороги мера сила физический ментальный менторский помощь ребятам начинающий разработчик выбрать путь профессиональный развития собрать роадмап штудирование книга стек фреймворках показывать практика черта нога сломит вариться неоднократно вебинар тема годы слово вебинар моды замениться воркшоп созрел учёт дыхание прочитать нетленка джефф уокер запуск созрелый сразу деньги итак ориентировочно воркшоп эфир объяснять целый небольшой агротех сервис сервис симулятор взрослый системы автоматизировать деятельность ферма выращиванию единорогов единорог строить единорожний конюшня обладать статами развивать скрещиванием закупать цветок нектар знают питаться единороги единорожий приплод продавать внутренний виртуальный рынке этакий небольшой бизнес симулятор целом исключение контекста с

In [30]:
df = filtered_df

---

In [31]:
docs = df['post_lemmatized'].tolist()

In [32]:
tfidf = TfidfVectorizer(min_df=10, max_df=0.9)

In [33]:
x = tfidf.fit_transform(docs)

In [34]:
tfidf.fit(docs)

TfidfVectorizer(max_df=0.9, min_df=10)

In [35]:
x.shape

(131, 64)

In [36]:
id2word = {i: token for token, i in tfidf.vocabulary_.items()}

**LDA**

In [37]:
n_topics = 10

In [38]:
lda = LatentDirichletAllocation(n_components=n_topics, random_state=31)

In [39]:
topics = lda.fit_transform(x)

In [40]:
lda.score(x)

-1974.1682405072193

In [41]:
topics.shape

(131, 10)

**Ключевые слова**

In [42]:
for i in range(n_topics):
    c = lda.components_[i, :]
    topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
    print("Тема",i+1, ":")
    print(", ".join(topic_tokens))
    print("\n ")

Тема 1 :
получить, развитие, разработка, команда, ментор, цель, система, особенно, возможность, обучение, информация, управление, английский, профессиональный, продукт, место, опытный, база, формат, программа

 
Тема 2 :
развитие, команда, материал, искать, сфера, уровень, наставник, управление, направление, курс, обучение, возможность, практический, инструмент, база, системный, создавать, персональный, разработка, приложение

 
Тема 3 :
управление, выбрать, рабочий, создавать, делиться, менторство, особенно, план, онлайн, идея, информация, помогать, учиться, опыт, процесс, менеджер, обучение, команда, итог, продукт

 
Тема 4 :
идея, опыт, навык, развитие, команда, курс, студент, решение, задание, продукт, возможность, помощь, интервью, сфера, программа, получить, тема, заниматься, особенно, уровень

 
Тема 5 :
развитие, обучения, ментор, поиск, идея, делиться, получить, сфера, рабочий, студент, интервью, выбрать, научиться, направление, задание, начало, обучение, онлайн, консультация,

**Типичные статьи**

In [43]:
for i in range(n_topics):
    doc_id = np.argmax(topics[:, i])
    print("Тема ", i)
    print(df.iloc[doc_id]["post"])
    print("\n")

Тема  0
Решил запустить хардкорную менторскую программу Product Army .Пока что 3 из 4 мест заполнены, цена дорогая, не набираю много людей, тк это впервые и очень хочу сфокусированно работать с только 4 амбициозными профессионалами..Это наша новая программа по Product Mentorship отлично подойдет как для новых специалистов, так и опытных профессионалов, которые развивают IT-продукты Старт: 20 мая⌛ Длительность: 10 недель Формат: 1-1 сессии, для Junior+ позиции или SwitchersЕсли есть заинтересованные, можете писать в лс, необязательно покупать, можно просто узнать 


Тема  1
Ты ищешь новую сферу деятельности?Ты готов к системной работе? Тебе нужно дать направление и сориентировать в мире IT? Погнали в IT с нами!Экспресс персональное обучение с наставником IT-профессии с нуляМы дадим теоретическую и (главное) практическую базу востребованной IT-профессии для возможности трудоустройства на уровне junoir в IT-компанииАналитика, Управление проектами, DevOps#управлениепроектами #аналитика #de